**Connecting to API**

In [ ]:
!pip install spotipy


# Import of libraries

import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


# Enter credentials from Spotify API

cid ='YOUR CID'
csecret ='YOUR SECRET CODE'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=csecret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

**Forming dataset**

In [5]:
# Playlists ids

play_lists = ["2NfTM2df5tHVUquNwet0yB", "0jH8HQYOmVYxHmb7prkgLb",
             "07iqzKaeKn1NsEScOzTmBU","1G8IpkZKobrIlXcVPoSIuf",
             "5S8SJdl1BDc0ugpkEvFsIL", "0z085jfTrSXxhFUyKKExxJ", 
             "7I6XVNQVayB46Zzh4GNPoU", "3WWv2APRYe9jR8L1YnK1Up",
             "29isGIzk5nnq9J9fnjDgD4", "6Vr7VftW8axlF8FzSjB5ff",
             "5xqpyfZyS1DVysoevdVyEn", "5OA5UAo3z5QnEzdiugiRBd",
             "4AeheqQLQs0kZMRpMRVkHI"]


# Function to extract data from a playlist 
# (thank to sfnxboy)


def get_playlist_data(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    results = tracks    

    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    playlist_artist_ids = []
    playlist_tracks_first_artists = []
    playlist_tracks_first_release_date = []
    playlist_tracks_popularity = []

    for i in range(len(results)):
        
        if i == 0:
            playlist_tracks_id = results[i]['track']['id']
            playlist_tracks_titles = results[i]['track']['name']
            playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
            playlist_tracks_popularity = results[i]['track']['popularity']
            
            artist_list = []
            for artist in results[i]['track']['artists']:
                artist_list= artist['name']
            playlist_tracks_artists = artist_list

            artist_id = []
            for artist in results[i]['track']['artists']:
                artist_id= artist['id']
            playlist_artist_ids = artist_id

            features = sp.audio_features(playlist_tracks_id)
            features_df = pd.DataFrame(data=features, columns=features[0].keys())
            features_df['title'] = playlist_tracks_titles
            features_df['all_artists'] = playlist_tracks_artists
            features_df['artist_id'] = playlist_artist_ids
            features_df['popularity'] = playlist_tracks_popularity
            features_df['release_date'] = playlist_tracks_first_release_date
            features_df = features_df[['id', 'title', 'all_artists', 'artist_id', 'popularity', 'release_date',
                                       'danceability', 'energy', 'key', 'loudness',
                                       'mode', 'acousticness', 'instrumentalness',
                                       'liveness', 'valence', 'tempo',
                                       'duration_ms', 'time_signature']]
            continue
        else:
            try:
                playlist_tracks_id = results[i]['track']['id']
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                playlist_tracks_popularity = results[i]['track']['popularity']
                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list= artist['name']
                playlist_tracks_artists = artist_list

                artist_id = []
                for artist in results[i]['track']['artists']:
                    artist_id= artist['id']
                playlist_artist_ids = artist_id
                features = sp.audio_features(playlist_tracks_id)
                new_row = {'id':[playlist_tracks_id],
               'title':[playlist_tracks_titles],
               'all_artists':[playlist_tracks_artists],
               'artist_id':[playlist_artist_ids],
               'popularity':[playlist_tracks_popularity],
               'release_date':[playlist_tracks_first_release_date],
               'danceability':[features[0]['danceability']],
               'energy':[features[0]['energy']],
               'key':[features[0]['key']],
               'loudness':[features[0]['loudness']],
               'mode':[features[0]['mode']],
               'acousticness':[features[0]['acousticness']],
               'instrumentalness':[features[0]['instrumentalness']],
               'liveness':[features[0]['liveness']],
               'valence':[features[0]['valence']],
               'tempo':[features[0]['tempo']],
               'duration_ms':[features[0]['duration_ms']],
               'time_signature':[features[0]['time_signature']]
               }

                dfs = [features_df, pd.DataFrame(new_row)]
                features_df = pd.concat(dfs, ignore_index = True)
            except:
                continue
                
    return features_df

def main(playlist):
  main_df = pd.DataFrame()
  for item in playlist:
      playlist_df = get_playlist_data('dmitry_gama', item)
      main_df = pd.concat([main_df, playlist_df], ignore_index = True)
  return main_df

union_df = main(play_lists)

In [ ]:
union_df

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6H7RUDjmjiGV4I9hE24Kk5,drivers license,jxdn,61,2021-01-19,0.598,0.787,10,-4.695,1,0.017500,0.000000,0.0887,0.298,144.021,237644,4
1,0we7ShV1o6cPTFjxOADPbC,Outcast,NF,59,2017-10-06,0.739,0.751,2,-7.161,1,0.165000,0.000000,0.0599,0.321,130.176,325720,4
2,6V81K1OcvYgHEnCA6A2Qx9,Start a Riot,Shaboozey,66,2018-12-14,0.728,0.727,2,-3.585,1,0.012800,0.000000,0.0882,0.890,143.144,171467,4
3,37PJvlIfYWqEX6fgjHlnTf,WTF Do I Know,Miley Cyrus,66,2020-11-27,0.564,0.849,4,-2.619,0,0.002550,0.000000,0.2690,0.634,157.905,171800,4
4,3b2ngmuuJP2XCUUwZvK9WQ,Every Window Is A Mirror,Joywave,41,2021-05-07,0.503,0.667,0,-4.166,1,0.001040,0.000438,0.0991,0.680,105.033,173227,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108896,7HIkQrTk0fvBvjahLTZtjB,Gimme Dat!,The Ambassador,30,2008-10-07,0.712,0.764,1,-5.834,0,0.196000,0.000000,0.0482,0.676,81.971,276440,4
108897,7JWiQAm5OMU4Qskjo0dm3e,It's Going Down,Canton Jones,38,2012-01-01,0.728,0.743,1,-6.180,0,0.000054,0.000000,0.1410,0.362,115.021,177973,4
108898,2ndr0bFApG3xvd2WCPLftz,Warmed Up,Swoope,25,2017-02-13,0.756,0.532,1,-7.054,1,0.033500,0.000000,0.1690,0.186,108.998,226239,4
108899,6oH5sjilDhOF7cZpkocOJc,One Day,Beleaf,39,2016-08-12,0.747,0.585,1,-5.942,0,0.087000,0.000000,0.0816,0.198,127.080,138553,4
